In [5]:
from lxml import etree

In [3]:
FULL_PAGE = ""
with open('./Class Schedule Listing.html') as f:
    FULL_PAGE = f.read()

In [6]:
parser = etree.HTMLParser()
tree = etree.fromstring(FULL_PAGE, parser)

In [11]:
tbody = tree.xpath('/html/body/div[3]/table[1]/tbody')[0]

In [33]:
class_head = tbody.xpath('tr[1]')[0]
class_body = tbody.xpath('tr[2]')[0]

In [16]:
class_head.xpath('th/a[1]/text()')

['Undergraduate Sophomore Seminar - 56742 - AAE 20000 - 001']

In [17]:
class_head.xpath('th/a[1]/@href')

['https://selfservice.mypurdue.purdue.edu/prod/bwckschd.p_disp_detail_sched?term_in=202010&crn_in=56742']

In [24]:
def parse_class_head(class_head):
    text = class_head.xpath('th/a[1]/text()')[0]
    url = class_head.xpath('th/a[1]/@href')[0]
    
    *title, crn, number, section = text.split('-')
    title = '-'.join(title).strip()
    crn = int(crn)
    number = number.strip()
    section = section.strip()
    
    return {
        'title': title,
        'crn': crn,
        'number': number,
        'section': section
    }

In [25]:
parse_class_head(class_head)

{'title': 'Undergraduate Sophomore Seminar',
 'crn': 56742,
 'number': 'AAE 20000',
 'section': '001'}

In [28]:
class_body.xpath('td/span[text()="Levels: "]/following-sibling::text()[1]')

['Graduate, Professional, Undergraduate \n']

In [35]:
def parse_class_body(class_body):
    levels = class_body.xpath('td/span[text()="Levels: "]/following-sibling::text()[1]')[0]
    levels = levels.split(',')
    levels = list(map(lambda x: x.strip(), levels))
    
    attributes = class_body.xpath('td/span[text()="Attributes: "]/following-sibling::text()[1]')[0]
    attributes = attributes.split(',')
    attributes = list(map(lambda x: x.strip(), attributes))
    
    return {
        'levels': levels,
        'attributes': attributes
    }

In [36]:
parse_class_body(class_body)

{'levels': ['Graduate', 'Professional', 'Undergraduate'],
 'attributes': ['Lower Division']}